<a href="https://colab.research.google.com/github/cdbm/airbnb-price-rediction-optimizer/blob/main/airbnb_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np 
import pandas as pd

# Database pre processing

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/cdbm/airbnb-price-rediction-optimizer/main/AB_NYC_2019.csv")
df.head()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [17]:
df = df.drop('name', axis=1)
df = df.drop('host_name', axis=1)
df = df.drop('last_review', axis=1)

In [18]:
df = df[df.price <= 500]

In [19]:
df.loc[df['price'] <= 50, 'ranges'] = '0_TO_50'
df.loc[(df['price'] > 50) & (df['price'] <= 100), 'ranges'] = '51_TO_100'
df.loc[(df['price'] > 100) & (df['price'] <= 150), 'ranges'] = '101_TO_150'
df.loc[(df['price'] > 150) & (df['price'] <= 200), 'ranges'] = '151_TO_200'
df.loc[(df['price'] > 200) & (df['price'] <= 250), 'ranges'] = '201_TO_250'
df.loc[(df['price'] > 250) & (df['price'] <= 300), 'ranges'] = '251_TO_300'
df.loc[(df['price'] > 300) & (df['price'] <= 350), 'ranges'] = '301_TO_350'
df.loc[(df['price'] > 350) & (df['price'] <= 400), 'ranges'] = '351_TO_400'
df.loc[(df['price'] > 400) & (df['price'] <= 450), 'ranges'] = '401_TO_450'
df.loc[(df['price'] > 450) & (df['price'] <= 500), 'ranges'] = '551_TO_500'

In [20]:
df['neighbourhood_group'] = df['neighbourhood_group'].astype('category')
df['neighbourhood'] = df['neighbourhood'].astype('category')
df['room_type'] = df['room_type'].astype('category')

In [21]:
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

In [22]:
df['ranges'] = df['ranges'].astype('category')

In [23]:
df['reviews_per_month'].fillna(0, inplace=True) 

In [24]:
df = df.drop('price', axis=1)


In [ ]:
df.head()

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df = df[:10000]
df.describe()

In [ ]:
df

In [28]:
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [29]:
  features = df.drop('ranges',axis=1)
  labels = df['ranges']

In [30]:
feature_train, feature_test, label_train, label_test = train_test_split(features, labels, test_size=0.2, random_state=1)

In [31]:
sc_X = StandardScaler()
feature_train=sc_X.fit_transform(feature_train)
feature_test=sc_X.fit_transform(feature_test)

# Genetic Algorithm

In [36]:
import random

In [37]:
POPULATION_SIZE = 10
GENERATIONS = 100
CHILDREN_RATE = 3

In [38]:
activation = ['identity', 'logistic', 'tanh', 'relu']
solver = ['lbfgs','sgd', 'adam']
learning_rate = ['constant', 'invscaling', 'adaptive']

In [39]:
def generate_individue():
  individue = {
      'activation':  activation[random.randint(0, 3)],
      'solver': solver[random.randint(0,2)],
      'learning_rate': learning_rate[random.randint(0,2)],
      'hidden_layer_sizes' : (100),
      'max_iter' : random.randint(100, 300),
      'fitness': 0  
  }

  individue['fitness'] = calculate_fitness(individue)
  return individue

In [48]:
def sortByFitness(e):
  return e['fitness']

In [50]:
def initiate_population(population_size):
  population = [[]] * population_size
  population = [generate_individue() for i in range(0, population_size)]
  population.sort(key=sortByFitness, reverse=True)
  return population

In [41]:
def choose_parents(population):
  father = random.randint(0, POPULATION_SIZE - 1)
  mother = random.randint(0, POPULATION_SIZE - 1)
  return [population[father], population[mother]]

In [49]:
def make_children(population):
  parents = choose_parents(population)
  child = {'activation': parents[random.randint(0,1)]['activation'],
          'hidden_layer_sizes': parents[random.randint(0,1)]['hidden_layer_sizes'],
          'learning_rate': parents[random.randint(0,1)]['learning_rate'],
          'max_iter': parents[random.randint(0,1)]['max_iter'],
          'solver': parents[random.randint(0,1)]['solver'],
          'fitness': 0}
 
  ##TODO - MUTATION
  child['fitness'] = calculate_fitness(child)
  population.append(child)
  population.sort(key=sortByFitness, reverse=True)
  return population[:10]

In [43]:
def calculate_fitness(individue):
  model_mlp = MLPClassifier(activation=individue['activation'],
                            hidden_layer_sizes=individue['hidden_layer_sizes'],
                            learning_rate=individue['learning_rate'],
                            max_iter=individue['max_iter'],
                            solver=individue['solver'])
  model_mlp.fit(feature_train, label_train) 

  prediction = model_mlp.predict(feature_test) 
  return accuracy_score(label_test, prediction)
  

Main


In [52]:
population = initiate_population(POPULATION_SIZE)
print(population)
for x in range(0, 10):
  population = make_children(population)
  print(population)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (186) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (261) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (286) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale 

[{'activation': 'logistic', 'solver': 'adam', 'learning_rate': 'invscaling', 'hidden_layer_sizes': 100, 'max_iter': 261, 'fitness': 0.478}, {'activation': 'tanh', 'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': 100, 'max_iter': 142, 'fitness': 0.4695}, {'activation': 'tanh', 'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': 100, 'max_iter': 286, 'fitness': 0.4675}, {'activation': 'relu', 'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': 100, 'max_iter': 104, 'fitness': 0.4645}, {'activation': 'logistic', 'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': 100, 'max_iter': 186, 'fitness': 0.447}, {'activation': 'identity', 'solver': 'lbfgs', 'learning_rate': 'invscaling', 'hidden_layer_sizes': 100, 'max_iter': 202, 'fitness': 0.445}, {'activation': 'identity', 'solver': 'lbfgs', 'learning_rate': 'invscaling', 'hidden_layer_sizes': 100, 'max_iter': 257, 'fitness': 0.445}, {'activation': 'relu', 'solver': 'lbfgs', '

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[{'activation': 'logistic', 'solver': 'adam', 'learning_rate': 'invscaling', 'hidden_layer_sizes': 100, 'max_iter': 261, 'fitness': 0.478}, {'activation': 'tanh', 'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': 100, 'max_iter': 142, 'fitness': 0.4695}, {'activation': 'tanh', 'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': 100, 'max_iter': 286, 'fitness': 0.4675}, {'activation': 'relu', 'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': 100, 'max_iter': 104, 'fitness': 0.4645}, {'activation': 'logistic', 'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': 100, 'max_iter': 186, 'fitness': 0.447}, {'activation': 'identity', 'solver': 'lbfgs', 'learning_rate': 'invscaling', 'hidden_layer_sizes': 100, 'max_iter': 202, 'fitness': 0.445}, {'activation': 'identity', 'solver': 'lbfgs', 'learning_rate': 'invscaling', 'hidden_layer_sizes': 100, 'max_iter': 257, 'fitness': 0.445}, {'activation': 'identity', 'hidden_layer_si

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (104) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (286) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[{'activation': 'logistic', 'solver': 'adam', 'learning_rate': 'invscaling', 'hidden_layer_sizes': 100, 'max_iter': 261, 'fitness': 0.478}, {'activation': 'tanh', 'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': 100, 'max_iter': 142, 'fitness': 0.4695}, {'activation': 'tanh', 'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': 100, 'max_iter': 286, 'fitness': 0.4675}, {'activation': 'relu', 'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': 100, 'max_iter': 104, 'fitness': 0.4645}, {'activation': 'tanh', 'hidden_layer_sizes': 100, 'learning_rate': 'adaptive', 'max_iter': 286, 'solver': 'adam', 'fitness': 0.4615}, {'activation': 'logistic', 'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': 100, 'max_iter': 186, 'fitness': 0.447}, {'activation': 'identity', 'solver': 'lbfgs', 'learning_rate': 'invscaling', 'hidden_layer_sizes': 100, 'max_iter': 202, 'fitness': 0.445}, {'activation': 'identity', 'solver': 'lbfgs', 'le

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (286) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[{'activation': 'logistic', 'solver': 'adam', 'learning_rate': 'invscaling', 'hidden_layer_sizes': 100, 'max_iter': 261, 'fitness': 0.478}, {'activation': 'logistic', 'hidden_layer_sizes': 100, 'learning_rate': 'adaptive', 'max_iter': 286, 'solver': 'adam', 'fitness': 0.471}, {'activation': 'tanh', 'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': 100, 'max_iter': 142, 'fitness': 0.4695}, {'activation': 'tanh', 'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': 100, 'max_iter': 286, 'fitness': 0.4675}, {'activation': 'relu', 'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': 100, 'max_iter': 104, 'fitness': 0.4645}, {'activation': 'tanh', 'hidden_layer_sizes': 100, 'learning_rate': 'adaptive', 'max_iter': 286, 'solver': 'adam', 'fitness': 0.4615}, {'activation': 'logistic', 'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': 100, 'max_iter': 186, 'fitness': 0.447}, {'activation': 'identity', 'solver': 'lbfgs', 'learn

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (286) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
